Back from holiday vacation and ready to finally get back to work! Oh, I have been looking forward to this!

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head()#Wooowwww that is a lot of data... still not technically 'big data' but it's still
#a bit intimidating.

(1117957, 22)


,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [5]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '')
df.columns

Index(['id', 'monsoonintensity', 'topographydrainage', 'rivermanagement',
       'deforestation', 'urbanization', 'climatechange', 'damsquality',
       'siltation', 'agriculturalpractices', 'encroachments',
       'ineffectivedisasterpreparedness', 'drainagesystems',
       'coastalvulnerability', 'landslides', 'watersheds',
       'deterioratinginfrastructure', 'populationscore', 'wetlandloss',
       'inadequateplanning', 'politicalfactors', 'floodprobability'],
      dtype='object')

In [6]:
#Hmm, is it sad that I'm already copying code I wrote? Regardless, I already like this approach...

for i in df.columns:
    if len(df[i].unique()) == df.shape[0]:
        df = df.drop([i], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
        print(f"Just dropped {i} as it wasn't adding any value whatsoever to our model.")
    else:
        print(f"Well, {i} has {len(df[i].unique())} unique fields, so we'll keep it...for now!")

Just dropped id as it wasn't adding any value whatsoever to our model.
Well, monsoonintensity has 17 unique fields, so we'll keep it...for now!
Well, topographydrainage has 19 unique fields, so we'll keep it...for now!
Well, rivermanagement has 17 unique fields, so we'll keep it...for now!
Well, deforestation has 18 unique fields, so we'll keep it...for now!
Well, urbanization has 18 unique fields, so we'll keep it...for now!
Well, climatechange has 18 unique fields, so we'll keep it...for now!
Well, damsquality has 17 unique fields, so we'll keep it...for now!
Well, siltation has 17 unique fields, so we'll keep it...for now!
Well, agriculturalpractices has 17 unique fields, so we'll keep it...for now!
Well, encroachments has 19 unique fields, so we'll keep it...for now!
Well, ineffectivedisasterpreparedness has 17 unique fields, so we'll keep it...for now!
Well, drainagesystems has 18 unique fields, so we'll keep it...for now!
Well, coastalvulnerability has 18 unique fields, so we'll 

Flood probability should hopefully be bounded between 0 and 1, definitionally. The rest all flutter between one of 17-20 fields. Interestings... seems like they'll be some nice correlation going on. Then again, perhaps the fields are just relative to the scoring metric, akin to flodprobablity being bounded between 0 and 100. Let us continue.

Speaking of which, let me bring up the original data dictionary: 